In [ ]:
import quackosm as qosm
import psutil
from pathlib import Path
from multiprocessing.pool import ThreadPool
import time
from pooch import retrieve
import tempfile
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

In [ ]:
def get_cpu_percentage() -> tuple[float, float]:
    """Get CPU usage."""
    return (psutil.cpu_percent(interval=None), round(time.time(), 2))


def get_memory_percentage() -> tuple[float, float]:
    """Get RAM usage."""
    return (psutil.virtual_memory().percent, round(time.time(), 2))


def get_directory_bytes_size(directory: Path) -> tuple[float, float]:
    """Get directory size in bytes."""
    while True:
        try:
            return (sum(f.stat().st_size for f in Path(directory).rglob("*")), round(time.time(), 2))
        except:
            pass

In [ ]:
get_cpu_percentage()

In [ ]:
get_memory_percentage()

In [ ]:
get_directory_bytes_size('files')

In [ ]:
def _execute_example(pbf_path, working_directory) -> None:
    print(
        qosm.convert_pbf_to_gpq(
            pbf_path=pbf_path, working_directory=working_directory, ignore_cache=True
        )
    )


def _sizeof_fmt(x, pos):
    if x < 0:
        return ""
    for x_unit in ["bytes", "kB", "MB", "GB", "TB"]:
        if x < 1024.0:
            return "%3.1f %s" % (x, x_unit)
        x /= 1024.0


def run_example(example_name: str, pbf_download_url: str) -> None:
    """Run example and monitor usage."""
    memory_values = []
    cpu_values = []
    disk_values = []

    downloaded_file = retrieve(url=pbf_download_url, known_hash=None)

    Path("files").mkdir(parents=True, exist_ok=True)

    with tempfile.TemporaryDirectory(dir="files") as tmp_dir_name:
        start_time = round(time.time(), 2)
        memory_values.append(get_memory_percentage())
        cpu_values.append(get_cpu_percentage())
        disk_values.append(get_directory_bytes_size(tmp_dir_name))

        time.sleep(0.1)

        memory_values.append(get_memory_percentage())
        cpu_values.append(get_cpu_percentage())
        disk_values.append(get_directory_bytes_size(tmp_dir_name))

        with ThreadPool() as pool:
            r = pool.apply_async(_execute_example, args=(downloaded_file, tmp_dir_name))
            while not r.ready():
                memory_values.append(get_memory_percentage())
                cpu_values.append(get_cpu_percentage())
                disk_values.append(get_directory_bytes_size(tmp_dir_name))
            r.get()

    time.sleep(0.1)

    memory_values.append(get_memory_percentage())
    cpu_values.append(get_cpu_percentage())
    disk_values.append(get_directory_bytes_size(tmp_dir_name))

    memory_values_adjusted = [(val, ts - start_time) for val, ts in memory_values]
    cpu_values_adjusted = [(val, ts - start_time) for val, ts in cpu_values]
    disk_values_adjusted = [(val, ts - start_time) for val, ts in disk_values]

    fig, ax = plt.subplots(3, sharex=True, figsize=(15, 10))

    # sns.set_theme(style="ticks")
    sns.set_theme(style="darkgrid")

    mem_val, mem_times = zip(*memory_values_adjusted)
    cpu_val, cpu_times = zip(*cpu_values_adjusted)
    dsk_val, dsk_times = zip(*disk_values_adjusted)

    sns.lineplot(x=mem_times, y=cpu_val, ax=ax[0])
    sns.lineplot(x=cpu_times, y=mem_val, ax=ax[1])
    sns.lineplot(x=dsk_times, y=dsk_val, ax=ax[2])

    pbf_file_size = Path(downloaded_file).stat().st_size
    ax[2].hlines(y=pbf_file_size, xmin=dsk_times[0], xmax=dsk_times[-1], color="r", linestyles="--")
    ax[2].yaxis.set_major_formatter(tkr.FuncFormatter(_sizeof_fmt))

    ax[0].set_ylabel("CPU usage %")
    ax[1].set_ylabel("Memory usage %")
    ax[2].set_ylabel("Disk usage")

    ax[2].set_xlabel("Time (s)")

    plt.suptitle(f"Resources usage - {example_name}.osm.pbf file")

    plt.tight_layout()

    plt.savefig(f"{example_name}.png", bbox_inches="tight")

In [ ]:
run_example("Monaco", "https://download.geofabrik.de/europe/monaco-latest.osm.pbf")